In [1]:
import pandas as pd
import numpy as np
import re
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# table1 基本信息表格

In [2]:
test2 = pd.read_excel(r"C:\Users\123\Desktop\jupyter notebook/2.test.xlsx", sheet_name='test2',header=0)

In [3]:
test2.head()

,No,样本编号,年龄,性别,统一病理格式,检测项目,检测方法,突变基因,氨基酸变化,核苷酸变化,突变类型
0,1,PC436,39,女,间变少突胶质瘤AO_III,6项,qPCR,TERT promotor,-,C250T,突变型
1,2,PC436,39,女,间变少突胶质瘤AO_III,6项,Sanger,TERT promotor,-,C250T,突变型
2,3,PC436,39,女,间变少突胶质瘤AO_III,6项,FINAL,TERT promotor,-,C250T,突变型
3,4,PC436,39,女,间变少突胶质瘤AO_III,6项,qPCR,TERT promotor,-,-,野生型
4,5,PC436,39,女,间变少突胶质瘤AO_III,6项,Sanger,TERT promotor,-,-,野生型


In [4]:
# 计算 性别	统一病理格式 的比例
def calpro(par):
    s1 = test2[par].value_counts()
    s2 = round(test2[par].value_counts(normalize=True)*100,2)
    s3 = s1.astype(str) + '(' + s2.astype(str) + '%)'
    temp = pd.DataFrame(s3)
    return temp

In [5]:
genpro = calpro('性别')
patpro = calpro('统一病理格式')

genpro
patpro

,性别
男,323(56.67%)
女,247(43.33%)


,统一病理格式
胶质母细胞瘤GBM_IV,190(33.33%)
星型细胞瘤A_II,133(23.33%)
少突胶质细胞瘤O_II,114(20.0%)
间变星型胶质瘤AA_III,57(10.0%)
间变少突胶质瘤AO_III,38(6.67%)
少突星形细胞瘤OA_II,19(3.33%)
胶质瘤(待定),19(3.33%)


In [6]:
# 计算 年龄 的比例
a = np.sum(test2['年龄'] >= 40)
b = np.sum(test2['年龄'] < 40)  
mean = test2['年龄'].mean()
std = test2['年龄'].std()

agepro = pd.DataFrame([str(mean)+ '+-' + str(round(std,2)),
                       str(b) + '(' + str(round(b/(a+b)*100,2)) + '%)',
                       str(a) + '(' + str(round(a/(a+b)*100,2)) + '%)'],
                      index=['均值+-标准差' ,'<40岁','>=40岁'])
agepro.columns = ['年龄']
agepro

,年龄
均值+-标准差,44.0+-13.9
<40岁,209(36.67%)
>=40岁,361(63.33%)


# -------------------------------------------------------
# table2 病理分型-基因突变表格
### 数据前处理，将'氨基酸变化', '核苷酸变化','甲基化','1p/19q' 整理为一项，便于后续分组统计。以病理分型为分组计算突变数量及比例，并合并所有病例分型的df

In [7]:
data = test2[['性别','年龄','样本编号','统一病理格式','突变基因','氨基酸变化','核苷酸变化','突变类型']]
data.head()

,性别,年龄,样本编号,统一病理格式,突变基因,氨基酸变化,核苷酸变化,突变类型
0,女,39,PC436,间变少突胶质瘤AO_III,TERT promotor,-,C250T,突变型
1,女,39,PC436,间变少突胶质瘤AO_III,TERT promotor,-,C250T,突变型
2,女,39,PC436,间变少突胶质瘤AO_III,TERT promotor,-,C250T,突变型
3,女,39,PC436,间变少突胶质瘤AO_III,TERT promotor,-,-,野生型
4,女,39,PC436,间变少突胶质瘤AO_III,TERT promotor,-,-,野生型


In [8]:
# 数据前处理

# 病理分型 WHO分组 列的获得
data[['病理分型','WHO分组']] = data['统一病理格式'].str.split('_',expand=True)

for i in range(len(data)):
    if not bool(re.search('[A-Z]',data['病理分型'][i])):
        continue
    else:
        data['病理分型'][i] = re.sub('[\u4e00-\u9fa4\0-9\,\。]','',data['病理分型'][i])

# 甲基化信息获得
result = data['突变类型'].str.match("^甲基化(.*)")
data['甲基化'] = '-'
data['甲基化'][result] = '甲基化'

# 1p 19q 1p19q 数据整理
data['1p/19q'] = '-'
data['1p/19q'][data['突变类型'].str.match("缺失")] = data[data['突变类型'].str.match("缺失")][data['突变基因'].str.match("^[0-9](.*)")]['突变基因'] + "缺失"
data = data.replace(["1p","19q","1p19q"],"1p/19q")

# 将'氨基酸变化', '核苷酸变化','甲基化','1p/19q' 整理为一项
data['变化'] = data[['氨基酸变化', '核苷酸变化','甲基化','1p/19q']].apply(lambda x: ''.join(x).replace('-',''), axis=1)

C:\Users\123\anaconda3\lib\site-packages\pandas\core\frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
<ipython-input-8-fb03c929dff1>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['病理分型'][i] = re.sub('[\u4e00-\u9fa4\0-9\,\。]','',data['病理分型'][i])
C:\Users\123\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3418: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

In [9]:
data

,性别,年龄,样本编号,统一病理格式,突变基因,氨基酸变化,核苷酸变化,突变类型,病理分型,WHO分组,甲基化,1p/19q,变化
0,女,39,PC436,间变少突胶质瘤AO_III,TERT promotor,-,C250T,突变型,AO,III,-,-,C250T
1,女,39,PC436,间变少突胶质瘤AO_III,TERT promotor,-,C250T,突变型,AO,III,-,-,C250T
2,女,39,PC436,间变少突胶质瘤AO_III,TERT promotor,-,C250T,突变型,AO,III,-,-,C250T
3,女,39,PC436,间变少突胶质瘤AO_III,TERT promotor,-,-,野生型,AO,III,-,-,
4,女,39,PC436,间变少突胶质瘤AO_III,TERT promotor,-,-,野生型,AO,III,-,-,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
565,男,42,HPA117,少突胶质细胞瘤O_II,BRAF,-,-,野生型,O,II,-,-,
566,男,42,HPA117,少突胶质细胞瘤O_II,1p/19q,-,-,缺失,O,II,-,1p缺失,1p缺失
567,男,42,HPA117,少突胶质细胞瘤O_II,1p/19q,-,-,缺失,O,II,-,19q缺失,19q缺失
568,男,42,HPA117,少突胶质细胞瘤O_II,1p/19q,-,-,缺失,O,II,-,1p19q缺失,1p19q缺失


In [30]:
pd.DataFrame(data[['病理分型','WHO分组']].value_counts())
pd.DataFrame(data['病理分型'].value_counts())
data['病理分型'].value_counts().sum()

,,0
病理分型,WHO分组,
GBM,IV,190
A,II,133
O,II,114
AA,III,57
AO,III,38
OA,II,19


,病理分型
GBM,190
A,133
O,114
AA,57
AO,38
OA,19
胶质瘤(待定),19


570

In [11]:
# 只筛选有突变的
data1=data[data['变化'].str.match(".+")]
data1

,性别,年龄,样本编号,统一病理格式,突变基因,氨基酸变化,核苷酸变化,突变类型,病理分型,WHO分组,甲基化,1p/19q,变化
0,女,39,PC436,间变少突胶质瘤AO_III,TERT promotor,-,C250T,突变型,AO,III,-,-,C250T
1,女,39,PC436,间变少突胶质瘤AO_III,TERT promotor,-,C250T,突变型,AO,III,-,-,C250T
2,女,39,PC436,间变少突胶质瘤AO_III,TERT promotor,-,C250T,突变型,AO,III,-,-,C250T
9,女,39,PC436,间变少突胶质瘤AO_III,IDH2,R172K,-,突变型,AO,III,-,-,R172K
10,女,39,PC436,间变少突胶质瘤AO_III,IDH2,R172K,-,突变型,AO,III,-,-,R172K
...,...,...,...,...,...,...,...,...,...,...,...,...,...
553,男,42,HPA117,少突胶质细胞瘤O_II,TERT promotor,-,C250T,突变型,O,II,-,-,C250T
566,男,42,HPA117,少突胶质细胞瘤O_II,1p/19q,-,-,缺失,O,II,-,1p缺失,1p缺失
567,男,42,HPA117,少突胶质细胞瘤O_II,1p/19q,-,-,缺失,O,II,-,19q缺失,19q缺失
568,男,42,HPA117,少突胶质细胞瘤O_II,1p/19q,-,-,缺失,O,II,-,1p19q缺失,1p19q缺失


In [12]:
df=pd.DataFrame()

# 获得不同病理下 不同突变基因的 不同变化的数量和百分比
for i in data['病理分型'].unique():
    
    # 数量：突变数
    d1 = pd.DataFrame(data1[data1['病理分型'] == i][['突变基因','变化','病理分型']].value_counts()).round(1)
    
    # 百分比：突变数/病理分型总数
    d2 = pd.DataFrame(data[data['病理分型'] == i][['突变基因','变化','病理分型']].value_counts()/data[data['病理分型'] == i]['病理分型'].value_counts()[i]).round(4)
    d2 = d2.applymap(lambda x:'%.2f%%' % (x*100))
    
    # df 合并
    d = pd.merge(d1,d2,on=['病理分型','突变基因','变化'],how='inner')
    d.columns = ['number','ratio']
    d = d.unstack(level=0).T
    df = pd.concat([df,d],axis=0)

# df 行列顺序修改    
df = df[['BRAF','IDH1','IDH2','TERT promotor','1p/19q','MGMT启动子甲基化']].unstack(level=0)
df = df.loc[['GBM','AA','AO','A','O','OA'],:]
df

突变基因   BRAF          IDH1           IDH2        TERT promotor                \
变化    V600E         R132H          R172K                C228T         C250T   
     number  ratio number   ratio number  ratio        number  ratio number   
病理分型                                                                          
GBM     NaN    NaN      3   1.58%    NaN    NaN             9  4.74%      9   
AA      NaN    NaN      6  10.53%    NaN    NaN           NaN    NaN    NaN   
AO      NaN    NaN      3   7.89%      3  7.89%             3  7.89%      3   
A         3  2.26%     12   9.02%    NaN    NaN             6  4.51%    NaN   
O       NaN    NaN     12  10.53%      3  2.63%             9  7.89%      9   
OA      NaN    NaN    NaN     NaN    NaN    NaN           NaN    NaN    NaN   

突变基因        1p/19q                                     MGMT启动子甲基化         
变化           19q缺失        1p19q缺失          1p缺失               甲基化         
      ratio number  ratio  number  ratio number  ratio     number  ratio  
病理分型                                                                      
GBM   4.74%      5  2.63%       1  0.53%      2  1.05%          6  3.16%  
AA      NaN    NaN    NaN     NaN    NaN      1  1.75%          2  3.51%  
AO    7.89%      2  5.26%       2  5.26%      1  2.63%          2  5.26%  
A       NaN    NaN    NaN     NaN    NaN    NaN    NaN          4  3.01%  
O     7.89%      6  5.26%       6  5.26%      6  5.26%          6  5.26%  
OA      NaN      1  5.26%       1  5.26%      1  5.26%          1  5.26%

In [17]:
# 同样，获得总的突变数目和百分比
d3 = pd.DataFrame(data1[['突变基因','变化']].value_counts()).round(1)
d4 = pd.DataFrame(data1[['突变基因','变化']].value_counts()/data[['突变基因','变化']].value_counts().sum()).round(4).applymap(lambda x:'%.2f%%' % (x*100))

d5 = pd.merge(d3,d4,on=['突变基因','变化'],how='inner')
d5.columns = ['number','ratio']
d5 = d5.T
d5 = d5[['BRAF','IDH1','IDH2','TERT promotor','1p/19q','MGMT启动子甲基化']]
d5

突变基因     BRAF   IDH1   IDH2 TERT promotor        1p/19q                 \
变化      V600E  R132H  R172K         C228T  C250T  19q缺失   1p缺失 1p19q缺失   
number      3     36      6            27     21     14     11      10   
ratio   0.53%  6.32%  1.05%         4.74%  3.68%  2.46%  1.93%   1.75%   

突变基因   MGMT启动子甲基化  
变化            甲基化  
number         21  
ratio       3.68%

# ------------------------------
# Figure3 画样本编号-突变基因_变化的热图
### 处理数据格式，将有突变的sample_id、mutgene_var与df_plot的行列名对映，存在则值为1，导出数据与注释信息，在R中用pheatmap画图

In [18]:
data2 = data1.copy()

In [19]:
# 年龄处理
data2.loc[data2['年龄'] < 40,'age'] = 'down40'
data2.loc[data2['年龄'] >= 40,'age'] = 'up40'
# 中英文替换处理
data2 = data2.replace(["1p缺失","19q缺失","1p19q缺失"],"DEL")
data2 = data2.replace("甲基化","Methy")
data2 = data2.replace("男","Man")
data2 = data2.replace("女","Woman")
data2 = data2.replace("MGMT启动子甲基化","MGMT Promoter")
# 拼接列
data2['Pathology'] = data2['病理分型'] + '_' + data2['WHO分组']
data2['mutgene_var'] = data2['突变基因'] + '_' + data2['变化']
# 行名修改
data2 = data2[['样本编号','mutgene_var','Pathology','性别','age']]
data2.rename(columns={'样本编号':'sample_id','性别':'sex'},inplace=True)

In [26]:
data2_plot = data2[['sample_id','mutgene_var']]

In [27]:
# 将有突变的行的 sample_id、mutgene_var 和 df_plot的行列名一一对映，存在则值为1，不存在为0，得到的 df_plot 是用于画图的数据
df_plot = pd.DataFrame(np.zeros((7,27)),columns=data2_plot['sample_id'].unique(),index=data2_plot['mutgene_var'].unique())
for i in range(len(data2_plot)):
    df_plot.loc[data2_plot.iloc[i][1],data2_plot.iloc[i][0]] = 1
df_plot

,PC436,PC168,PC507,PC290,PC858,PC876,PC458,PC615,PC011,PB910,...,PB771,PB981,PC043,PC249,PC923,PB606,PC055,PC078,PC047,HPA117
TERT promotor_C250T,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
IDH2_R172K,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1p/19q_DEL,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
MGMT Promoter_Methy,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,...,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0
TERT promotor_C228T,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
IDH1_R132H,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0
BRAF_V600E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [28]:
# 得到注释信息
df_ann = data2[['sample_id','Pathology','sex','age']]
df_ann = df_ann[~df_ann.duplicated()]

In [29]:
# 数据导出
df_ann.to_csv('data_ann.csv')
df_plot.to_csv('data_plot.csv')

### R 画图
```
data_plot <- read.csv('./data_plot.csv',row.names = 1)
data_ann <- read.csv('./data_ann.csv')


library(pheatmap)
# 注释信息
annotation_col = data.frame(Age = data_ann$age , Sex = data_ann$sex , Pathology = data_ann$Pathology) 
rownames(annotation_col) = data_ann$sample_id
# 画热图
pheatmap(data_plot,color = colorRampPalette(c("white", "white", "darkgreen"))(50), legend = FALSE,
         ,cluster_row = FALSE,cluster_col = FALSE, annotation_col = annotation_col, angle_col = "45") 

```